# Наивный байесовский классификатор

1. Собрать датасет из любых двух открытых групп социальной сети VK (не менее 1500 непустых сообщений из каждой группы). Вывести (или сохранить во внешний файл) собранный датасет.

2. Разделить датасет на обучающую и тестовую выборки.

3. Попробовать различные методы предобработки текста (для лемматизации текстов на русском языке можyо попробовать библиотеку `pymorphy2` или  аналоги).

4. Используя наивный байесовский классификатор, научиться отличать тематику сообщений. Экспортировать словари в pkl файл.

5. Проверить результат классификации на тестовой выборке.

6. Написать отчет об этапах предподготовки данных и обучении модели. Описать последовательность работы над корпусом текстов для достижения лучшей точности классификации.



In [1]:
import numpy as np
import joblib

class Naive_bayes():

    def __init__(self):
        self.classes_cnt = {} # class_num: [dataset_class_counter, total_features_in_class_counter]
        self.freq = {}        # (feature, class_num): counter
        self.unique = set()   # unique features in dataset
        self.fitted = False

    def fit(self, dataset):
        for features, label in dataset:
            if label not in self.classes_cnt:
                self.classes_cnt[label] = [0, 0]
            self.classes_cnt[label][0] += 1
            for feature in features:
                if (feature, label) not in self.freq:
                    self.freq[(feature, label)] = 0
                self.freq[(feature, label)] += 1
                self.classes_cnt[label][1] += 1
                self.unique.add(feature)
        self.fitted = True

    def predict(self, features, alpha=1):
        if not self.fitted: return None
        dataset_cnt = 0
        for label in self.classes_cnt:
            dataset_cnt += self.classes_cnt[label][1]

        return max(self.classes_cnt.keys(),
                   key = lambda cl:
                   np.log10(self.classes_cnt[cl][0]/dataset_cnt) +
                   sum(np.log10((alpha + self.freq.get((feature, cl), 0))/\
                           (alpha*len(self.unique) + self.classes_cnt[cl][1]))
                       for feature in features))

    def export_model(self, path='dicts.pkl'):
        model = self.classes_cnt, self.freq, self.unique
        joblib.dump(model, path)
        return model

    def import_model(self, path='dicts.pkl'):
        self.classes_cnt, self.freq, self.unique = joblib.load(path)
        self.fitted = True

In [2]:
%%capture
!pip install nltk
!pip install pymorphy2

!pip install py-linq

In [3]:
posts_separator = '\n––––––––––\n'


def read_posts(filename):
    with open(filename, 'r', encoding='utf8') as file:
        return file.read().split(posts_separator)

In [4]:
posts0 = read_posts('itis_posts.txt')
print(len(posts0))
print(posts0[0])

2005
Директор ИТИС Михаил Абрамский выступит на очередной серии научно-популярного проекта PROНаука в КФУ 24 марта!💥


In [5]:
posts1 = read_posts('strategium_posts.txt')
len(posts1)

2039

In [6]:
import random

random.seed(20)

random.shuffle(posts0)
random.shuffle(posts1)
print(posts0[0])

Наши дорогие ИТИСовцы, и будущие ИТИСовцы!

Поздравляем ВАС с днем Всех Влюбленных!!!
Будьте очень счастливы, самими собой во  всем. Веселых ВАМ вечеров с семьей, с друзьями, веселых поездок и походов в кино, в кафе, конечно же на пары!!!)) Крепких общих побед над всеми трудностями жизни! Желаем вам крепкую, большую, дружную семью, в которой даже в мелочах один за всех и все за одного!﻿

В любви признавайтесь открыто.

Не думайте, сколько Вам лет.

Условности будут забыты. 

Границ для прекрасного нет.


In [7]:
from py_linq import Enumerable

In [8]:
def posts_to_words(posts):
    return (
        Enumerable(posts).select(lambda x: x.replace('\n', ' '))  # Replaces new lines with spaces
                         .select(lambda x: x.replace('_', ' '))  # Replaces underscores with spaces
                         .select(lambda x: x.split(' '))          # Split
                         .to_list()
    )

In [9]:
import nltk.corpus

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("russian")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\artemgur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
from pymorphy2 import MorphAnalyzer

morph = MorphAnalyzer()

In [11]:
import re


def extract_word(s):
    # Regex which extracts true words.
    # Doesn't match one-letter words, but these are stopwords anyway
    match = re.match(r'[a-zа-яё]+-?[a-zа-яё]+', s)
    return match.group() if match else ''

In [12]:
def post_clear_noise(post: list[str]):
    return (
        Enumerable(post).select(lambda x: x.lower())                   # Lowercase
                        .where(lambda x: not ('http://' in x) and not ('https://' in x))  # Removes URLs
                        .select(extract_word)
                        .select(lambda x: morph.normal_forms(x)[0])    # Lemmatization
                        .where(lambda x: x)                            # Removes empty strings
                        .where(lambda x: x not in stopwords)           # Removes stopwords
                        .to_list()
    )

def posts_clear_noise(posts):
    return list(map(post_clear_noise, posts))

In [13]:
posts0_preprocessed = posts_clear_noise(posts_to_words(posts0))
posts0_preprocessed[0]

['наш',
 'дорогой',
 'итисовца',
 'будущий',
 'итисовца',
 'поздравлять',
 'день',
 'весь',
 'влюбить',
 'очень',
 'счастливый',
 'весь',
 'весёлый',
 'вечер',
 'семья',
 'друг',
 'весёлый',
 'поездка',
 'поход',
 'кино',
 'кафе',
 'пара',
 'крепкий',
 'общий',
 'победа',
 'весь',
 'трудность',
 'жизнь',
 'желать',
 'крепкий',
 'больший',
 'дружный',
 'семья',
 'который',
 'мелочь',
 'весь',
 'всё',
 'любовь',
 'признаваться',
 'открыто',
 'думать',
 'сколько',
 'год',
 'условность',
 'забытый',
 'граница',
 'прекрасный']

In [14]:
posts1_preprocessed = posts_clear_noise(posts_to_words(posts1))

In [15]:
def posts_add_labels(posts, label):
    return list(map(lambda x: [x, label], posts))

In [16]:
train_size = 1500

posts0_train = posts0_preprocessed[:train_size]
posts0_test = posts0_preprocessed[train_size:]

posts1_train = posts1_preprocessed[:train_size]
posts1_test = posts1_preprocessed[train_size:]

In [17]:
bayes = Naive_bayes()
bayes.fit(posts_add_labels(posts0_train, 0) + posts_add_labels(posts1_train, 1))

In [18]:
len(bayes.freq)

21583

In [19]:
def bayes_predict_many(bayes, posts):
    return list(map(bayes.predict, posts))

In [20]:
test_labels0 = [0] * len(posts0_test)
test_labels1 = [1] * len(posts1_test)
test_labels = test_labels0 + test_labels1

prediction0 = bayes_predict_many(bayes, posts0_test)
prediction1 = bayes_predict_many(bayes, posts1_test)
prediction = prediction0 + prediction1

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

In [22]:
print('Accuracy:', accuracy_score(test_labels, prediction))
print('Accuracy on file 0:', accuracy_score(test_labels0, prediction0))
print('Accuracy on file 1:', accuracy_score(test_labels1, prediction1))

Accuracy: 0.9712643678160919
Accuracy on file 0: 0.9900990099009901
Accuracy on file 1: 0.9536178107606679


In [23]:
print('Precision:', precision_score(test_labels, prediction))
print('Recall:', recall_score(test_labels, prediction))
print('AUC ROC:', roc_auc_score(test_labels, prediction))

Precision: 0.9903660886319846
Recall: 0.9536178107606679
AUC ROC: 0.971858410330829


In [24]:
error_posts0 = Enumerable(posts0).zip(Enumerable(prediction0)).where(lambda x: x[1] == 1).select(lambda x: x[0])
for post in error_posts0:
    print(post)
    print(posts_separator)

#ИТИС_анонс #ИТИС_лагерь 
Ты школьник и мечтаешь научиться программировать роботов? Приглашаем тебя на Робо-лагерь в ИТИС КФУ с 29 октября по 3 ноября! Подробности в сюжете ниже! 
Стоимость смены: 10 000 руб (обед и экскурсии включены) 
⏰Режим: понедельник (29.10)-суббота (3.11) с 9.00 до 18.00 
📖Посмотреть программу: https://kpfu.ru/itis/v-osennie-kanikuly-itis-provedet.. 
✍🏻Записаться: https://itiskfu.timepad.ru/event/828188/ 
☎Контакты: juliya@it.kfu.ru 
8 (843) 221-34-33 (доб.12)

––––––––––

Высшая школа ИТИС и фирма 1С объявляет о проведении регионального отбора на конкурс "Умник". Победитель получит 200 000 рублей на реализацию проекта. 
Успей подготовить презентацию и подать заявку на участие до 29 октября. Подробнее здесь —->http://umnik.fasie.ru/1c/

http://umnik.fasie.ru/1c/page/about-program.html

––––––––––

#ИТИС_абитуриент2020 #ITISinternational 
В КФУ опубликованы приказы о зачислении иностранных студентов
В приказе перечислены иностранные студенты бакалавриата и специа

In [27]:
len(error_posts0)

5

In [25]:
error_posts1 = Enumerable(posts1).zip(Enumerable(prediction1)).where(lambda x: x[1] == 0).select(lambda x: x[0])
for post in error_posts1:
    print(post)
    print(posts_separator)

Разработчики Hitman(!!!) делают фэнтезийную RPG.

Пока нет абсолютно никакой инфы, кроме той, что после релиза игру будут постоянно обновлять. Видимо, примерно как трилогию Hitman.

Рабочее название игры - Project Fantasy. 

#rpg@strategium

––––––––––

Продолжаем рассказывать подробности о #Dune: Spice Wars - помимо домов [https://vk.com/@strategium-dom-harkonnen-v-dune-spice-wars|Харконнен] и [https://vk.com/@strategium-dom-atreides-v-dune-spice-wars|Атрейдес] вы сможете выбрать фракцию Контрабандистов. Подробнее об особенностях фракции и её юнитах - в статье.

Другие новости и информация по игре — по тегу #DuneSpiceWars@strategium

––––––––––

Разработчики [https://vk.com/wall-31092576_11053|Knights of Honor II: Sovereign] — масштабной средневековой стратегии в режиме реального времени — посвятили новый дневник ИИ.

При разработке одной их основных целей было обучить ИИ вести себя как живому игроку. Так, во время войны ИИ не будет просто собирать всё армию и атаковать. Он будет усил

In [28]:
len(error_posts1)

25

In [26]:
bayes.export_model()

({0: [1500, 56030], 1: [1500, 101858]},
 {('наш', 0): 346,
  ('дорогой', 0): 63,
  ('итисовца', 0): 13,
  ('будущий', 0): 13,
  ('поздравлять', 0): 118,
  ('день', 0): 322,
  ('весь', 0): 305,
  ('влюбить', 0): 1,
  ('очень', 0): 43,
  ('счастливый', 0): 13,
  ('весёлый', 0): 6,
  ('вечер', 0): 9,
  ('семья', 0): 10,
  ('друг', 0): 62,
  ('поездка', 0): 5,
  ('поход', 0): 1,
  ('кино', 0): 4,
  ('кафе', 0): 3,
  ('пара', 0): 16,
  ('крепкий', 0): 6,
  ('общий', 0): 36,
  ('победа', 0): 54,
  ('трудность', 0): 3,
  ('жизнь', 0): 58,
  ('желать', 0): 82,
  ('больший', 0): 26,
  ('дружный', 0): 7,
  ('который', 0): 299,
  ('мелочь', 0): 1,
  ('всё', 0): 192,
  ('любовь', 0): 2,
  ('признаваться', 0): 1,
  ('открыто', 0): 1,
  ('думать', 0): 11,
  ('сколько', 0): 5,
  ('год', 0): 393,
  ('условность', 0): 1,
  ('забытый', 0): 4,
  ('граница', 0): 4,
  ('прекрасный', 0): 12,
  ('ирина', 0): 20,
  ('максимов', 0): 16,
  ('заместитель', 0): 31,
  ('директор', 0): 146,
  ('высокий', 0): 581,
 